In [15]:
import pandas as pd 
#from textblob import TextBlob
#from textblob.sentiments import NaiveBayesAnalyzer

from tqdm import tqdm 
tqdm.pandas()
import os 

In [16]:
df = pd.concat([pd.read_csv(f'13b/entire_sep29_{i}.csv') for i in [0, 1, 2, 3, 4, 5, 'extras']])

In [17]:
# replace image path parent dir with /share/ju/nexar_data/training_datasets/street_flooding/all_no_letterboxing 
df['image_path'] = df['image_path'].apply(lambda x: os.path.join('/share/ju/nexar_data/training_datasets/street_flooding/all_no_letterboxing', x.split('/')[-1]))

In [18]:
df.columns

Index(['Unnamed: 0', 'image_path', 'q1', 'q2', 'response_1', 'response_2',
       'Unnamed: 0.1'],
      dtype='object')

In [19]:
def get_sentiment(text):
    blob = TextBlob(text, analyzer=NaiveBayesAnalyzer())
    return blob.sentiment[0]


In [20]:
def quick_sentiment(text):
    # if first word is yes, 1
    # if first word is no, 0
    # else, 0.5
    text = str(text).lower()
    # strip lspace
    text = text.lstrip()
    if text.startswith('yes'):
        return 1
    elif text.startswith('no'):
        return 0
    else:
        return 0

In [21]:
for i in range(1,3):
    df[f'sentiment_{i}'] = df[f'response_{i}'].progress_apply(quick_sentiment)

100%|██████████| 926618/926618 [00:00<00:00, 1559338.69it/s]


In [22]:
df.dropna(subset=['response_1'], inplace=True)
df.drop(columns=['Unnamed: 0'], inplace=True)

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 926212 entries, 0 to 405
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   image_path    926212 non-null  object 
 1   q1            926212 non-null  object 
 2   q2            926212 non-null  object 
 3   response_1    926212 non-null  object 
 4   response_2    926212 non-null  object 
 5   Unnamed: 0.1  406 non-null     float64
 6   sentiment_1   926212 non-null  int64  
 7   sentiment_2   926212 non-null  int64  
dtypes: float64(1), int64(2), object(5)
memory usage: 63.6+ MB


In [24]:
df['sentiment_1'].value_counts()

sentiment_1
0    924747
1      1465
Name: count, dtype: int64

In [25]:
df['sentiment_2'].value_counts()

sentiment_2
0    790669
1    135543
Name: count, dtype: int64

In [26]:
# copy the sentiment_1 images to DIR 
DIR = "./high_conf_images"
os.makedirs(DIR, exist_ok=True)

for i, row in df[df['sentiment_1'] == 1].iterrows():
    pass
    #os.system(f'cp {row["image_path"]} {DIR}/{row["image_path"].split("/")[-1]}')


In [27]:
df

,image_path,q1,q2,response_1,response_2,Unnamed: 0.1,sentiment_1,sentiment_2
0,/share/ju/nexar_data/training_datasets/street_...,Does this image show more than a foot of stand...,Is the street in this image flooded?,"No, the image does not show more than a foot o...","No, the street in the image does not appear to...",NaN,0,0
1,/share/ju/nexar_data/training_datasets/street_...,Does this image show more than a foot of stand...,Is the street in this image flooded?,"No, the image does not show more than a foot o...","Yes, the street in the image appears to be flo...",NaN,0,1
2,/share/ju/nexar_data/training_datasets/street_...,Does this image show more than a foot of stand...,Is the street in this image flooded?,"No, the image does not show more than a foot o...","No, the street in the image does not appear to...",NaN,0,0
3,/share/ju/nexar_data/training_datasets/street_...,Does this image show more than a foot of stand...,Is the street in this image flooded?,"No, the image does not show more than a foot o...","No, the street in the image is not flooded. Th...",NaN,0,0
4,/share/ju/nexar_data/training_datasets/street_...,Does this image show more than a foot of stand...,Is the street in this image flooded?,"No, the image does not show more than a foot o...",The street in the image does not appear to be ...,NaN,0,0
...,...,...,...,...,...,...,...,...
401,/share/ju/nexar_data/training_datasets/street_...,Does this image show more than a foot of stand...,Is the street in this image flooded?,"No, the image does not show more than a foot o...",The street in the image does not appear to be ...,4.0,0,0
402,/share/ju/nexar_data/training_datasets/street_...,Does this image show more than a foot of stand...,Is the street in this image flooded?,"No, the image does not show more than a foot o...","Yes, the street appears to be flooded, as evid...",3.0,0,1
403,/share/ju/nexar_data/training_datasets/street_...,Does this image show more than a foot of stand...,Is the street in this image flooded?,"No, the image does not show more than a foot o...","No, the street in the image does not appear to...",2.0,0,0
404,/share/ju/nexar_data/training_datasets/street_...,Does this image show more than a foot of stand...,Is the street in this image flooded?,"No, the image does not show more than a foot o...","No, the street in the image is not flooded. Th...",1.0,0,0


In [28]:
df.to_csv('entire_sep29_all_13b.csv', index=False)